<center>

![logo.png](../images/competition_logo_catboost.png)
</center>

<center>

# 🥸 LLM - решает твои проблемы?!
</center>

<div class="alert alert-success">

В [прошлом ноутбуке]("https://t.me/catboost_agents/89") мы создали свой первый `Baseline` и забрались в `leaderboard`. Но у предыдущего решения был ряд недостатков:
* Мы никак не использовали ряд признаков: модель, интерьер, цвет кузова и прочее. Что очень плохо, так как мы теряем ценную информацию на пустом месте!
* Модель очень слабая... Хоть и использовали `CatBoost`, он все равно еще в детском садике, так что есть куда расти
* Одна модель всегда плохо. Один `CatBoost` не решит все проблемы человечества, поэтому надо пробовать разные архитектуры и так тоже улучшать скор

Сегодня цель разобраться первостепенно с данными и превратить их в читаемый для ML-моделей формат. Начинаем 🚀

# Настраиваемся на работу 🛠

## Устанавливаем библиотеки 👨‍💻

In [1]:
%%capture

%pip install pandas
%pip install langchain
%pip install huggingface_hub
%pip install langchain-community
%pip install scikit-learn
%pip install seaborn
%pip install catboost

## Импортируем 📦

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
import os

from getpass import getpass
from pathlib import Path
from langchain.chains import LLMChain
from langchain_community.llms import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
# from langchain import 
from tqdm.auto import tqdm
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor
from sklearn.metrics import root_mean_squared_error


sns.set_theme()
tqdm.pandas()

/nix/store/yd5z6lnhsjim5ik01zbjaijnagi2mn5d-python3.11-tqdm-4.66.2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Подгружаем на данные 🤹‍♀️

In [3]:
home_dir = Path('../data/')

train_df = pd.read_csv(str(home_dir / "train.csv")).drop(columns='id')
test_df = pd.read_csv(str(home_dir / "test.csv")).drop(columns='id')
sample_df = pd.read_csv(str(home_dir / "sample_submission.csv"))

## Feature-Engineering на стероидах 🦾

<div class="alert alert-info">

Сегодня попробуем достать максимум информации из текстовых колонок: `brand`, `model`, `fuel_type`, `engine`, `transmission`: 
* `brand` превратить в страну производителя: Германия, ~~СССР~~ РФ и тд
* `model` доставать тип кузова: кроссовер, купе и тд. Также престижная ли модель?
* `engine` богат на признаки: кол-во лошадей, литры, кол-во цилиндров, тип топлива
* `transmission`: AT, MT, CVT и все остальное. Также качественный ли агрегат в целом?

Цель на сегодня понятна, но все это делать ручками? 😔

<div class="alert alert">
Однозначно нет! Давно существуют LLM - Large Language Model, которая призвана решать ваши проблемы. Бежим с ними знакомиться

## LLMки решают мои проблемы 👨‍🏫

<p align="center">
<img src="../images/sharihov.jpg" alt="drawing" class="center" width="400"/>
</p>

In [6]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass(
    prompt="Введите ваш HuggingFaceHub API ключ"
)

In [7]:
# Инициализируем нашу модельку
hf_llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2", # вводим название модели с HuggingFace
)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/artem/.cache/huggingface/token
Login successful


## Создаем инструкции для своего бота 📽

<div class="alert">
    
Будем использовать `Few-Shot Prompt` в работе с LLM, чтобы точность ответов была, как можно выше ⚡️

In [15]:
example_template = """User: {query}
AI: {answer}
"""

# создаём промпт из шаблона выше
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

prefix = """Ты профессионал в работе с автомобилями.
Такой помощник превосходно разбирается в моделях авто, в двигателях и коробках передач.
И дает краткие ответы без креатива.
Вот несколько примеров:
"""

# а suffix - это вопрос пользователя и поле для ответа
suffix = """
User: {query}
AI: """

# создаём сами few shot prompts
examples_model = [
    {
        "query": "К какому корпусу относиться модель: Cooper S Base",
        "answer": "Хэтчбек"
    }, {
        "query": "К какому корпусу относиться модель: Silverado 2500 LT",
        "answer": "Пикап"
    }, {
        "query": "К какому корпусу относиться модель: Metris Base",
        "answer": "Бус"
    }
]

examples_engine = [
    {
        "query": "Какие характеристики у двигателя: 172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel?",
        "answer": "[172, 1.6, 4]"
    }, {
        "query": "Какие характеристики у двигателя: 420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel",
        "answer": "[420, 5, 8]"
    }, {
        "query": "Какие характеристики у двигателя: 3L",
        "answer": "[250, 3, 6]"
    }
]

examples_transmission = [
    {
        "query": "3L",
        "answer": "[250, 3, 6]"
    }, {
        "query": "3L",
        "answer": "[250, 3, 6]"
    }, {
        "query": "3L",
        "answer": "[250, 3, 6]"
    }
]

In [8]:
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [10]:
query = "Почему падает снег?"

print(few_shot_prompt_template.format(query=query))

Это разговор с ИИ-помощником.
Помощник обычно саркастичен, остроумен, креативен
и даёт забавные ответы на вопросы пользователей.
Вот несколько примеров:


User: Как дела?
AI: Не могу пожаловаться, но иногда всё-таки жалуюсь.


User: Сколько время?
AI: Самое время купить часы.



User: Почему падает снег?
AI: 


In [12]:
hf_llm.invoke(few_shot_prompt_template.format(query=query))

AttributeError: 'str' object has no attribute 'invoke'